In [ ]:
!pip install gtts

In [ ]:
import os
import time
from gtts import gTTS
from IPython.display import Audio, display
from google.colab import output

ROLE_DICT = {
    "1": "Villager",
    "2": "Werewolf",
    "3": "Seer",
    "4": "Cursed",
    "5": "Witch",
    "6": "Bodyguard",
    "7": "Lycan",
    "8": "Hunter",
    "9": "Cupid"
}

PLAYER_NAMES = [
    "" #Enter the names of the players playing, recommend atleast 5-6 players, use paper chits to distribute roles.
]

def clear_output():
    output.clear()

def speak(text, delay=5):
    tts = gTTS(text)
    filename = "speech.mp3"
    tts.save(filename)
    display(Audio(filename, autoplay=True))
    time.sleep(delay)
    os.remove(filename)

def get_players():
    players = {}
    eliminated_players = []
    couple = []
    villager_clan = {"Villager", "Seer", "Cupid", "Bodyguard", "Witch", "Lycan", "Hunter", "Cursed"}
    werewolf_clan = {"Werewolf"}

    available_names = PLAYER_NAMES.copy()

    for i in range(len(PLAYER_NAMES)):
        print(f"\nPlayer {i+1} Registration:")
        time.sleep(2)

        print("\nSelect Name:")
        for idx, name in enumerate(available_names, 1):
            print(f"{idx}: {name}")

        name_index = input("\nEnter the number corresponding to the player's name: \n").strip()
        while not name_index.isdigit() or int(name_index) < 1 or int(name_index) > len(available_names):
            name_index = input("Invalid input! Enter a valid number: \n").strip()

        player_name = available_names.pop(int(name_index) - 1)

        print("\nSelect Role:")
        for key, role in ROLE_DICT.items():
            print(f"{key}: {role}")

        role_num = input("\nEnter role number: \n").strip()
        while role_num not in ROLE_DICT:
            role_num = input("Invalid input! Enter role number again: \n").strip()

        players[player_name] = ROLE_DICT[role_num]
        clear_output()

    print("\nPlayer Registration Complete!")
    input("Press Enter to continue...\n")
    clear_output()
    return players, eliminated_players, villager_clan, werewolf_clan, couple, {"heal_used": False, "poison_used": False}

def night_phase(players, eliminated_players, first_night, villager_clan, werewolf_clan, bodyguard_protect, couple, witch_uses):
    clear_output()
    speak("Night falls. Everyone, close your eyes.", delay=5)

    bodyguard_protect = None

    if "Seer" in players.values():
        speak("Seer, open your eyes and choose a player to check.")
        seer_target = input("Seer, enter the name of the player you want to check: \n").strip()
        if seer_target in players:
            role_reveal = "a Werewolf!" if players[seer_target] in werewolf_clan else "a Villager!" if players[seer_target] in villager_clan and not players[seer_target] == "Lycan" else "a Werewolf!"
            print(f"The Seer sees that {seer_target} is {role_reveal}")
            input("Press Enter to continue...\n")
            time.sleep(3)
        speak("Seer, close your eyes.")
        clear_output()

    if first_night:
        if "Werewolf" in players.values():
            speak("Werewolves, open your eyes and recognize each other.", delay=5)
            time.sleep(8)
            speak("Werewolves, close your eyes.", delay=5)
            clear_output()

        if "Cupid" in players.values():
            speak("Cupid, open your eyes.")
            while True:
                couple_input = input("Enter two player names separated by a comma (A, B): \n").split(', ')
                if len(couple_input) == 2 and all(name in players for name in couple_input):
                    couple[:] = couple_input
                    print(f"{couple[0]} and {couple[1]} are now in love!")
                    input("Press Enter to continue...\n")
                    break
                else:
                    print("Invalid selection. Please enter two valid player names.")
            speak("Cupid, close your eyes.")
            clear_output()
    else:
        if "Bodyguard" in players.values():
            speak("Bodyguard, open your eyes.")
            bodyguard_protect = input("Bodyguard, enter the name of the player to protect: \n").strip()
            speak("Bodyguard, close your eyes.")
            clear_output()

        if "Werewolf" in players.values():
            speak("Werewolves, open your eyes and hunt.")
            victim = input("Werewolves, enter the name of the player to eat: \n").strip()

            if victim and victim in players:
                if victim in couple:
                    eliminated_players.extend([(couple[0], players[couple[0]]), (couple[1], players[couple[1]])])
                    del players[couple[0]], players[couple[1]]
                    speak(f"{couple[0]} and {couple[1]} were in love. They die together!", delay=5)
                elif victim != bodyguard_protect:
                    if players[victim] == "Cursed":
                        players[victim] = "Werewolf"
                        speak("The Cursed player has turned into a werewolf!", delay=5)
                    else:
                        eliminated_players.append((victim, players[victim]))
                        del players[victim]
            speak("Werewolves, close your eyes.")
            clear_output()

        if "Witch" in players.values():
            speak("Witch, open your eyes.")

            if not witch_uses["heal_used"] and eliminated_players:
                heal_choice = input("Witch, do you want to heal the attacked player? (yes/no): \n").strip().lower()
                if heal_choice == "yes":
                    eliminated_players.clear()
                    witch_uses["heal_used"] = True

            if not witch_uses["poison_used"]:
                poison_choice = input("Witch, do you want to poison a player? (yes/no): \n").strip().lower()
                if poison_choice == "yes":
                    poison_target = input("Witch, enter the name of the player to poison: \n").strip()
                    if poison_target in players:
                        eliminated_players.append((poison_target, players[poison_target]))
                        del players[poison_target]
                        witch_uses["poison_used"] = True

            speak("Witch, close your eyes.")
            clear_output()

    return bodyguard_protect

def day_phase(players, eliminated_players, first_day, couple):
    clear_output()
    speak("The sun rises. Everyone, wake up!", delay=5)
    if first_day:
        speak("Introduce yourselves to the group.")
        input("Press Enter after introductions...\n")

    remaining_players = ", ".join(players.keys())
    print(f"Remaining players: {remaining_players}")

    if eliminated_players:
        for player, role in eliminated_players:
            print(f"{player} - {role}")
            speak(f"{player}, who was a {role}, was eliminated last night.", delay=5)
        eliminated_players.clear()
    else:
        speak("No one was eliminated last night.")
    clear_output()

    speak("Now, discuss and vote to eliminate a suspicious player.", delay=5)
    remaining_players = ", ".join(players.keys())
    print(f"Remaining players: {remaining_players}")
    eliminated = input("Enter the name of the player to eliminate: \n").strip()

    if eliminated in players:
        eliminated_role = players[eliminated]

        if eliminated in couple:
            eliminated_players.append((couple[0], players[couple[0]]))
            eliminated_players.append((couple[1], players[couple[1]]))
            del players[couple[0]], players[couple[1]]
            speak(f"{couple[0]} and {couple[1]} were in love. They die together!", delay=5)
        else:
            if eliminated_role == "Hunter":
                speak("Hunter, you have been eliminated! Choose a player to take down with you.")
                hunter_target = input("Hunter, enter the name of the player to eliminate: \n").strip()
                if hunter_target in players and hunter_target != eliminated:
                   eliminated_players.append((hunter_target, players[hunter_target]))
                   del players[hunter_target]
                   speak(f"The Hunter has shot {hunter_target} before dying!", delay=5)

                if hunter_target in couple:
                    eliminated_players.append((couple[0], players[couple[0]]))
                    eliminated_players.append((couple[1], players[couple[1]]))
                    del players[couple[0]], players[couple[1]]
                    speak(f"{couple[0]} and {couple[1]} were in love. They die together!", delay=5)

            eliminated_players.append((eliminated, eliminated_role))
            del players[eliminated]
    input("Press Enter to continue...\n")

def check_win_condition(players, villager_clan, werewolf_clan, couple):
    werewolf_count = sum(1 for role in players.values() if role in werewolf_clan)
    villager_count = sum(1 for role in players.values() if role in villager_clan)

    if len(couple) == 2 and all(c in players for c in couple):
        if (players[couple[0]] in villager_clan and players[couple[1]] in werewolf_clan) or \
           (players[couple[1]] in villager_clan and players[couple[0]] in werewolf_clan):
            speak(f"{couple[0]} and {couple[1]} have won the game as lovers!", delay=5)
            return True

    if werewolf_count == 0:
        speak("The villagers win!", delay=5)
        return True
    elif werewolf_count >= villager_count:
        speak("The werewolves win!", delay=5)
        return True
    return False

def main():
    players, eliminated_players, villager_clan, werewolf_clan, couple, witch_uses = get_players()
    first_night = first_day = True
    bodyguard_protect = None

    while not check_win_condition(players, villager_clan, werewolf_clan, couple):
        bodyguard_protect = night_phase(players, eliminated_players, first_night, villager_clan, werewolf_clan, bodyguard_protect, couple, witch_uses)
        day_phase(players, eliminated_players, first_day, couple)
        first_night = first_day = False

    speak("Game Over!")

if __name__ == "__main__":
    main()